In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium # Data Visualization
import matplotlib.pyplot as plt # Data Visualization
import seaborn as sns # Data Visualization (Themes)
import geopandas as gpd # Interactive Map with folium
import plotly.express as px

# Input data files are available in the read-only "../input/" directory

#Start by presssing (Shift+Enter)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv


# Youtube Data Analysis

Hey all I'm just starting with python anlysis, all feedback is welcomed and appreciated!

**What's my objective?**
* Top 10 youtube channels (location, total count of views, channel creation date, category)
* Top youtube category for maximum number of views
* Create interactive geographical graph


In [2]:
# The defualt encoding can't read the file, therefore change to latin-1
df = pd.read_csv('/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv', encoding = 'latin-1')

#Change the index to rank, this could also be done during .read_csv()
df.set_index('rank', inplace = True)

In [3]:
# Grouping by country to see the sum of subscribers for each country - the top 5 
total_subscribers_country = df.groupby('Country')['subscribers'].sum().sort_values(ascending=False)
total_subscribers_country.head()

# It seems that even though

Country
United States     7329500000
India             4310600000
Brazil            1221800000
United Kingdom     917400000
Mexico             626900000
Name: subscribers, dtype: int64

Analyze how many youtubers in each country: 
* There are a total of 995 Youtube accounts (See if there are any duplicates)

In [4]:
df['Youtuber'].unique()
# I see not all characters are encoded properly - tried different encoders but the symbols persisted...
# Note: Some of the characters are emojis

array(['T-Series', 'YouTube Movies', 'MrBeast',
       'Cocomelon - Nursery Rhymes', 'SET India', 'Music',
       'ýýý Kids Diana Show', 'PewDiePie', 'Like Nastya', 'Vlad and Niki',
       'Zee Music Company', 'WWE', 'Gaming', 'BLACKPINK', 'Goldmines',
       'Sony SAB', '5-Minute Crafts', 'BANGTANTV', 'Sports',
       'Justin Bieber', 'HYBE LABELS', 'Zee TV',
       "Pinkfong Baby Shark - Kids' Songs & Stories", 'Canal KondZilla',
       'ChuChu TV Nursery Rhymes & Kids Songs', 'Shemaroo Filmi Gaane',
       'Colors TV', 'T-Series Bhakti Sagar', 'Dude Perfect', 'Movieclips',
       'Tips Official', 'El Reino Infantil', 'Wave Music', 'Aaj Tak',
       'Sony Music India', 'EminemMusic', 'Marshmello', 'YRF',
       "LooLoo Kids - Nursery Rhymes and Children's Songs", 'Ed Sheeran',
       'Infobells - Hindi', 'Taylor Swift', 'Ariana Grande',
       'BillionSurpriseToys  - Nursery Rhymes & Cartoons', 'JuegaGerman',
       'Billie Eilish', 'Get Movies', 'Shemaroo', 'Badabun',
       'SonyMu

In [5]:
# Overview of all of the countries in the dataset
df['Country'].unique()

array(['India', 'United States', nan, 'Japan', 'Russia', 'South Korea',
       'United Kingdom', 'Canada', 'Brazil', 'Argentina', 'Chile', 'Cuba',
       'El Salvador', 'Pakistan', 'Philippines', 'Thailand', 'Colombia',
       'Barbados', 'Mexico', 'United Arab Emirates', 'Spain',
       'Saudi Arabia', 'Indonesia', 'Turkey', 'Venezuela', 'Kuwait',
       'Jordan', 'Netherlands', 'Singapore', 'Australia', 'Italy',
       'Germany', 'France', 'Sweden', 'Afghanistan', 'Ukraine', 'Latvia',
       'Switzerland', 'Vietnam', 'Malaysia', 'China', 'Iraq', 'Egypt',
       'Andorra', 'Ecuador', 'Morocco', 'Peru', 'Bangladesh', 'Finland',
       'Samoa'], dtype=object)

# Top 10 Youtubers
In the following section I focus on only the top 10. Afteerwards I will complete an overview with all of the youtubers

4 - Kids Channels <br>
3 - Music Channels <br>
2 - American Personality Channels <br>
1 - Movie Channel <br>


In [6]:
#Want to condense to the information I want to display by selecting fewer columns in new dataframe
# Top 10 Youtube Accounts - the year they were created, their category, 

selected_columns = ['Youtuber','subscribers','category','created_year','Country','Latitude', 'Longitude']
top_10_youtubers = df[selected_columns].head(10)
top_10_youtubers

,Youtuber,subscribers,category,created_year,Country,Latitude,Longitude
rank,,,,,,,
1,T-Series,245000000,Music,2006.0,India,20.593684,78.962880
2,YouTube Movies,170000000,Film & Animation,2006.0,United States,37.090240,-95.712891
3,MrBeast,166000000,Entertainment,2012.0,United States,37.090240,-95.712891
4,Cocomelon - Nursery Rhymes,162000000,Education,2006.0,United States,37.090240,-95.712891
5,SET India,159000000,Shows,2006.0,India,20.593684,78.962880
6,Music,119000000,NaN,2013.0,NaN,NaN,NaN
7,ýýý Kids Diana Show,112000000,People & Blogs,2015.0,United States,37.090240,-95.712891
8,PewDiePie,111000000,Gaming,2010.0,Japan,36.204824,138.252924
9,Like Nastya,106000000,People & Blogs,2016.0,Russia,61.524010,105.318756


In [7]:
# Testing out the folium library to create geospatial map
map_osm = folium.Map(location = [20.593684, 78.962880], zoom_start = 5)
tooltip="Indian Youtubers"
folium.Marker([20.593684, 78.962880], popup="<i>Total 2: <br> T-Series, SET India</i>", tooltip=tooltip).add_to(map_osm)

map_osm

In [8]:
import pandas as pd
import plotly.express as px

aggregated_youtubers = top_10_youtubers.groupby(['Country','Latitude', 'Longitude'])['Youtuber'].apply(', '.join).reset_index()

fig = px.scatter_geo(aggregated_youtubers, lat = 'Latitude', lon='Longitude', text='Country', hover_name='Youtuber', color = 'Country',
                                                    template='ggplot2')

fig.show()

# Overall Youtubers

Interesting how the categories range.